In [1]:
from matplotlib import pyplot as plt

import numpy as np

import tensorflow as tf
from tensorflow import keras

from keras.models import load_model

import fedhex as fx
from fedhex.pretrain import generation as fxgen
from fedhex.train import SelectiveProgbarLogger, BatchLossHistory
from fedhex.train.tf import loss_MADE

KeyboardInterrupt: 

In [ ]:
ndim = 2
nx = ny = 10
sigmax = 1 / (nx + 1) / 2
sigmay = 1 / (ny + 1) / 2
sigma = np.array([sigmax, sigmay])

gen = fx.GridGaussGenerator(cov_strat=fxgen.RepeatStrategy(fxgen.DiagCov(ndim=ndim, sigma=sigma)),
                      ngausx=nx,
                      ngausy=ny,
                      seed=0x2024)

nsamp = 1000
samp, lab = gen.generate(nsamp=nsamp)
lablocs = np.unique(lab, axis=0)

In [ ]:
nmade = 8
ninputs = ncinputs = ndim
hidden_units = [16, 16]
batch_size = 1 << 12
end_epoch = 20


log_freq = 10
flow_path = "model/2024.06.07-gg"
callbacks = []
callbacks.append(SelectiveProgbarLogger(1, epoch_interval=log_freq, epoch_end=end_epoch))
callbacks.append(BatchLossHistory(flow_path + "/loss.npy"))


model = fx.MADEManager(nmade=nmade,
               ninputs=ninputs,
               ncinputs=ncinputs,
               hidden_units=hidden_units,
               activation="relu")

opt = keras.optimizers.Adam(learning_rate=1e-3)
loss = loss_MADE

model.compile_model(opt=opt, loss=loss)

model.train_model(dm=gen,
                  batch_size=batch_size,
                  end_epoch=end_epoch,
                  callbacks=callbacks,
                  path=flow_path)

In [ ]:
mm = fx.MADEManager.import_model(flow_path)

In [ ]:
end_epoch *= 2
callbacks = []
callbacks.append(SelectiveProgbarLogger(1, epoch_interval=log_freq, epoch_end=end_epoch))
callbacks.append(BatchLossHistory(flow_path + "/loss.npy"))
mm.train_model(dm=gen,
                  batch_size=batch_size,
                  starting_epoch=end_epoch/2,
                  end_epoch=end_epoch,
                  callbacks=callbacks,
                  path=flow_path)